### Quick Description

This notebook calculates the toxicity score from each sentence in a subtitles' file. Finally, it persists the data in a DataFrame like output.

In [1]:
import yaml
import glob
import os
import pandas as pd
import time
import re
import logging
import sys

from tqdm import tqdm
from flair.models import TextClassifier
from flair.data import Sentence


sys.path.append("../utils")
from toxicity_api_communication import get_toxicity_score

In [2]:
filepaths = yaml.load(open("../config/filepaths.yaml"))
credentials = yaml.load(open("../config/credentials.yaml"))["perspective-api"]

/tmp/ipykernel_166383/1089545284.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  filepaths = yaml.load(open("../config/filepaths.yaml"))
/tmp/ipykernel_166383/1089545284.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credentials = yaml.load(open("../config/credentials.yaml"))["perspective-api"]


In [3]:
def calculate_toxicity_scores(credentials, save_dir):
    classifier = TextClassifier.load('en-sentiment')
    filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(save_dir, "*"))]

    for filename in tqdm(filenames, total=len(filenames)):
        df_sentences = pd.read_csv(os.path.join(save_dir, filename))
        for i, row in df_sentences.iterrows():
            sentence = Sentence(row.text)
            classifier.predict(sentence)

            df_sentences.loc[i, "flair_score"] = (
                1 - sentence.labels[0]._score
                if sentence.labels[0]._value == "POSITIVE"
                else 1 + sentence.labels[0]._score
            )
            
        # Because 2 is the absolute maximum value
        df_sentences["flair_score"] = (
            (df_sentences["flair_score"] - df_sentences["flair_score"].min())
            / (2 - df_sentences["flair_score"].min())
        )

        df_sentences.to_csv(os.path.join(save_dir, filename), index=False)

In [4]:
calculate_toxicity_scores(
    credentials,
    filepaths["scored_data"]
)

2022-05-25 22:46:38,973 loading file /home/guilherme/.flair/models/sentiment-en-mix-distillbert_4.pt


100%|███████████████████████████████████████████████████████████████| 634/634 [18:54<00:00,  1.79s/it]
